In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gmaps
import os
import requests
import json
from config import api_key

base_url = "https://maps.googleapis.com/maps/api/geocode/json"
county_file= os.path.join('..','Resources','county_vote_df.csv')
region_file= os.path.join('..','Resources','region_vote_df.csv')

In [2]:
county_vote_df = pd.read_csv(county_file)

In [3]:
county_vote_df["Lat"] = ""
county_vote_df["Lng"] = ""
county_vote_df.head()

,Unnamed: 0,Region,Area,Valid_Votes,Remain,Leave,Pct_Remain,Pct_Leave,Lat,Lng
0,0,East,Peterborough,87392,34176,53216,0.391066,0.608934,,
1,1,East,Luton,84481,36708,47773,0.434512,0.565488,,
2,2,East,Southend-on-Sea,93870,39348,54522,0.419175,0.580825,,
3,3,East,Thurrock,79916,22151,57765,0.277179,0.722821,,
4,4,East,Bedford,86066,41497,44569,0.482153,0.517847,,


In [4]:
non_Eng = ['Wales', 'Scotland', 'Northern Ireland']

In [5]:
params = {"key":api_key}

for index, row in county_vote_df.iterrows():
    if row['Region'] not in non_Eng:
        if row['Area'] == 'Herefordshire, County of':
            city = "Herefordshire County, United Kingdom"
        else:
            city = "{0}, United Kingdom".format(row['Area'])
        params["address"] = city
        areas_lat_lng = requests.get(base_url, params=params)
        
        areas_lat_lng = areas_lat_lng.json()
        
        try:
            county_vote_df.loc[index, "Lat"] = areas_lat_lng["results"][0]["geometry"]["location"]["lat"]
            county_vote_df.loc[index, "Lng"] = areas_lat_lng["results"][0]["geometry"]["location"]["lng"]
        except (KeyError, IndexError):
            print("Missing field/result... skipping.")
    else: 
        city = row['Area']
        params["address"] = city
        areas_lat_lng = requests.get(base_url, params=params)
        
        areas_lat_lng = areas_lat_lng.json()
        
        try:
            county_vote_df.loc[index, "Lat"] = areas_lat_lng["results"][0]["geometry"]["location"]["lat"]
            county_vote_df.loc[index, "Lng"] = areas_lat_lng["results"][0]["geometry"]["location"]["lng"]
        except (KeyError, IndexError):
            print("Missing field/result... skipping.")
            
county_vote_df.head(10)

,Unnamed: 0,Region,Area,Valid_Votes,Remain,Leave,Pct_Remain,Pct_Leave,Lat,Lng
0,0,East,Peterborough,87392,34176,53216,0.391066,0.608934,52.5695,-0.24053
1,1,East,Luton,84481,36708,47773,0.434512,0.565488,51.8787,-0.420025
2,2,East,Southend-on-Sea,93870,39348,54522,0.419175,0.580825,51.5459,0.707712
3,3,East,Thurrock,79916,22151,57765,0.277179,0.722821,51.4935,0.35292
4,4,East,Bedford,86066,41497,44569,0.482153,0.517847,52.136,-0.466655
5,5,East,Central Bedfordshire,158804,69670,89134,0.438717,0.561283,52.003,-0.465139
6,6,East,Cambridge,57799,42682,15117,0.738456,0.261544,52.2053,0.121817
7,7,East,East Cambridgeshire,48086,23599,24487,0.490767,0.509233,52.4037,0.242812
8,8,East,Fenland,52626,15055,37571,0.286075,0.713925,52.5745,0.0487678
9,9,East,Huntingdonshire,99927,45729,54198,0.457624,0.542376,52.4037,-0.242812


In [7]:
locations = county_vote_df[["Lat", "Lng"]].astype(float)
for_brexit = county_vote_df["Pct_Leave"].astype(float)

In [8]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=for_brexit, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

# Adjust heat_layer setting to help with heatmap dissipating on zoom
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 1

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))